# Heat Exchanger NTU Unit Model with Ideal & IAPWS Property Package

![](heat_exchanger_4.svg)

**Problem Statement**: In this example, we will be transfering heat from hot to cold streams of MEA (monoethylamine), simulating heat integration of an aqueous MEA system.

**Hot Side Inlet**

Flow Rate = 60.54879 mol/s

Mole fraction (CO2) = 0.0158

Mole fraction (H2O) = 0.8747

Mole fraction (MEA) = 0.1095

Pressure = 202650 Pa

Temperature = 392.23 K

**Cold Side Inlet**

Flow Rate = 63.01910 mol/s

Mole fraction (CO2) = 0.0414

Mole fraction (H2O) = 0.8509

Mole fraction (MEA) = 0.1077

Pressure = 202650 Pa

Temperature = 326.36 K

This example will demonstrate the simulation of the NTU heat exchanger by fixing the following degrees of freedom:
- heat transfer area
- heat transfer coefficient
- effectiveness
- hot and cold side pressure changes


IDAES documentation reference for heat exchanger 1D model: https://idaes-pse.readthedocs.io/en/latest/technical_specs/model_libraries/generic/unit_models/heat_exchanger_NTU.html

**Setting up the problem in IDAES**

In [1]:
# Import pyomo package 
from pyomo.environ import ConcreteModel, SolverFactory, Constraint, value, units

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

#import the MEA property package to create a properties block for the flowsheet
from idaes.power_generation.carbon_capture.mea_solvent_system.properties.MEA_solvent \
    import configuration as aqueous_mea

from idaes.generic_models.properties.iapws95 import htpx

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

#Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
from idaes.core.util.model_statistics import degrees_of_freedom

#Import a heat exchanger unit
from idaes.generic_models.unit_models.heat_exchanger_ntu import HeatExchangerNTU as HXNTU

#Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

# Steady State Model
m.fs = FlowsheetBlock(default={"dynamic": False})

# Setup property packages for hot and cold side
# MEA property package for hotside
m.fs.hotside_properties = GenericParameterBlock(default=aqueous_mea)

# MEA property package for coldside
m.fs.coldside_properties = GenericParameterBlock(default=aqueous_mea)

In the [0D Heat Exchanger model example](http://localhost:8888/notebooks/GitHub/examples-pse/src/Examples/UnitModels/Operations/heat_exchanger_0D_testing.ipynb), the inlets are specified exactly and the solver determines heat transfer based on resulting temperatures. Here, we specify hot and cold streams and mandate that heat flow from the hot to the cold stream using the transfer unit method.

In [2]:
#Create an instance of the heat exchanger unit, attaching it to the flowsheet
#Specify that the property package to be used with the heater is the one we created earlier.
m.fs.heat_exchanger = HXNTU(default={
            "hot_side": {"property_package": m.fs.hotside_properties,
                         "has_pressure_change": True},
            "cold_side": {"property_package": m.fs.coldside_properties,
                          "has_pressure_change": True}})

# Call the degrees_of_freedom function, get initial DOF
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))



The initial DOF is 15


In [3]:
assert DOF_initial == 15

In [4]:
# Hot fluid
m.fs.heat_exchanger.hot_inlet.flow_mol[0].fix(60.54879) # mol/s
m.fs.heat_exchanger.hot_inlet.temperature[0].fix(392.23) # K
m.fs.heat_exchanger.hot_inlet.pressure[0].fix(202650) # Pa
m.fs.heat_exchanger.hot_inlet.mole_frac_comp[0, "CO2"].fix(0.0158) # dimensionless
m.fs.heat_exchanger.hot_inlet.mole_frac_comp[0, "H2O"].fix(0.8747) # dimensionless
m.fs.heat_exchanger.hot_inlet.mole_frac_comp[0, "MEA"].fix(0.1095) # dimensionless

# Cold fluid
m.fs.heat_exchanger.cold_inlet.flow_mol[0].fix(63.01910) # mol/s
m.fs.heat_exchanger.cold_inlet.temperature[0].fix(326.36) # K
m.fs.heat_exchanger.cold_inlet.pressure[0].fix(202650) # Pa
m.fs.heat_exchanger.cold_inlet.mole_frac_comp[0, "CO2"].fix(0.0414) # dimensionless
m.fs.heat_exchanger.cold_inlet.mole_frac_comp[0, "H2O"].fix(0.8509) # dimensionless
m.fs.heat_exchanger.cold_inlet.mole_frac_comp[0, "MEA"].fix(0.1077) # dimensionless

DOF_initial = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_initial))


The DOF is 3


### Option 1: Fix HTC and dimensions of each domain


In [5]:
# Unit design variables
m.fs.heat_exchanger.area.fix(100) # m2
m.fs.heat_exchanger.heat_transfer_coefficient.fix(200) # W/m2/K
m.fs.heat_exchanger.effectiveness.fix(0.7) # dimensionless

m.fs.heat_exchanger.hot_side.deltaP.fix(-2000) # Pa
m.fs.heat_exchanger.cold_side.deltaP.fix(-2000) # Pa

# Call the degrees_of_freedom function, get final DOF
DOF_final = degrees_of_freedom(m)
print("The DOF is {0}".format(DOF_final))

The DOF is 0


In [6]:
assert DOF_final == 0

In [7]:
#Initialize the flowsheet, and set the output at WARNING
m.fs.heat_exchanger.initialize()

#Solve the simulation using ipopt
#Note: If the degrees of freedom = 0, we have a square problem
opt = SolverFactory('ipopt')
solve_status = opt.solve(m, tee = True)

#Display a readable report
m.fs.heat_exchanger.report()

2022-03-03 13:47:08 [INFO] idaes.init.fs.heat_exchanger.hot_side.properties_in: Starting initialization
2022-03-03 13:47:08 [INFO] idaes.init.fs.heat_exchanger.hot_side.properties_in: Property initialization: optimal - Optimal Solution Found.
2022-03-03 13:47:08 [INFO] idaes.init.fs.heat_exchanger.hot_side.properties_out: Starting initialization
2022-03-03 13:47:09 [INFO] idaes.init.fs.heat_exchanger.hot_side.properties_out: Property initialization: optimal - Optimal Solution Found.
2022-03-03 13:47:09 [INFO] idaes.init.fs.heat_exchanger.hot_side: Initialization Complete
2022-03-03 13:47:09 [INFO] idaes.init.fs.heat_exchanger.cold_side.properties_in: Starting initialization
2022-03-03 13:47:09 [INFO] idaes.init.fs.heat_exchanger.cold_side.properties_in: Property initialization: optimal - Optimal Solution Found.
2022-03-03 13:47:09 [INFO] idaes.init.fs.heat_exchanger.cold_side.properties_out: Starting initialization
2022-03-03 13:47:09 [INFO] idaes.init.fs.heat_exchanger.cold_side.prope

In [8]:
from pyomo.opt import TerminationCondition, SolverStatus
import pytest

# Check if termination condition is optimal
assert solve_status.solver.termination_condition == TerminationCondition.optimal
assert solve_status.solver.status == SolverStatus.ok

assert value(m.fs.heat_exchanger.hot_outlet.temperature[0]) == pytest.approx(364.31, abs=1e-2)
assert value(m.fs.heat_exchanger.cold_outlet.temperature[0]) == pytest.approx(357.93, abs=1e-2)
